In [1]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install scholarly

Requirement already up-to-date: pip in /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages (19.2.3)


In [2]:
!{sys.executable} -m pip install nltk

!{sys.executable} -m pip install sklearn

!{sys.executable} -m pip install stemming

In [1]:
import scholarly
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from stemming.porter2 import stem

In [2]:
sq = input('What do you want to search on Google Schoalr?')

What do you want to search on Google Schoalr?automatic drum


In [3]:
search_query = scholarly.search_pubs_query(sq)

# create title_list
t = []
# create abstract_list
a = []

# get google scholar title and abstract
limit = 30
for index, item in enumerate(search_query,1):
    # a.append('PaperID: ' + str(index) + 'Title: ' + item.bib['title'] + 'Abstract: ' + item.bib['abstract'])
    print('PaperID: ' + str(index))
    t.append(item.bib['title'])
    print('Title: ' + item.bib['title'])
    if 'abstract' in item.bib:
        a.append(item.bib['abstract'])
        print('Abstract: ' + item.bib['abstract'])
    else:
        a.append('none')
        print('Abstract: ' + 'none')
    if index == limit:
        break

PaperID: 1
Title: Automatic drum transcription and source separation
Abstract: While research has been carried out on automated polyphonic music transcription, to-date the problem of automated polyphonic percussion transcription has not received the same degree of attention. A related problem is that of sound source separation, which attempts to …
PaperID: 2
Title: Automatic Drum Sound Description for Real-World Music Using Template Adaptation and Matching Methods.
Abstract: This paper presents an automatic description system of drum sounds for real-world musical audio signals. Our system can represent onset times and names of drums by means of drum descriptors defined in the context of MPEG-7. For their automatic description, drum sounds …
PaperID: 3
Title: Automatic Drum Transcription Using Bi-Directional Recurrent Neural Networks.
Abstract: Automatic drum transcription (ADT) systems attempt to generate a symbolic music notation for percussive instruments in audio recordings. Neural 

In [4]:
# show selected paper title
x = [int(x) for x in input("Enter 5 to 10 PaperID with space between each other: ").split()]
print("The paper you choose are: ")
for index, item in enumerate(x):
    print(t[item-1])

Enter 5 to 10 PaperID with space between each other: 3 5 13
The paper you choose are: 
Automatic Drum Transcription Using Bi-Directional Recurrent Neural Networks.
On automatic drum transcription using non-negative matrix deconvolution and itakura saito divergence
Drum transcription with non-negative spectrogram factorisation


In [5]:
# get selected abstract
sa =[]
for index, item in enumerate(x):
    sa.append(a[item-1])
#print(sa) 

# get abstract excluded from selected item
x.sort(reverse=True)
for index,item in enumerate(x):
    a.pop(item-1)
    t.pop(item-1)
#print(a)
#print(t)

# make non-selected articles a data frame for calculation
ta_df = pd.DataFrame({'Title': t, 'Abstract': a})

In [6]:
ta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 2 columns):
Title       27 non-null object
Abstract    27 non-null object
dtypes: object(2)
memory usage: 512.0+ bytes


In [7]:
# combine 2 lists to one
s = " "
sa = s.join(sa)

ca = a
ca.append(sa)

In [8]:
# Stopwords
my_additional_stop_words =["author","and","of","the","research","\n"]
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

# STEM
ca = [[stem(word)for word in sentence.split(" ")]for sentence in ca]
#print(ca)

# from lists of list to strings of list
y = len(ca)
for i in range(0,y):
    ca[i]=" ".join(ca[i])

# build tf-idf matrix
tfidf = text.TfidfVectorizer(input=ca,stop_words=stop_words,analyzer='word',lowercase=True)
matrix=tfidf.fit_transform(ca)
print(matrix.shape)
#matrix.shape

# calculate similarity score
sim_unigram = cosine_similarity(matrix)
# print(type)

(28, 402)


In [9]:
type(sim_unigram)

numpy.ndarray

In [10]:
# build function to get similar score for all papers by locating them
def get_similar_papers(x):
    return ta_df.loc[np.argsort(-x)]

recomPaper = get_similar_papers(sim_unigram[-1])
#print(recomPaper)

sim_unigram[-1][::-1].sort() #descending
# sim_unigram[-1].sort() #ascending

#print(sim_unigram[-1])

recomPaper["similar_score"]=sim_unigram[-1]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
recomPaper.head(10)

,Title,Abstract,similar_score
27,NaN,NaN,1.000000
12,Automatic Drum Transcription for Polyphonic Re...,Automatic drum transcription is the process of...,0.292948
1,Automatic Drum Sound Description for Real-Worl...,This paper presents an automatic description s...,0.251745
4,ENST-Drums: an extensive audio-visual database...,"… Keywords: Research database, Automatic drum ...",0.235338
13,Exploration of techniques for automatic labeli...,We report on the progress of current work rega...,0.203814
20,Automatic Drum Transcription Using the Student...,Automatic drum transcription is a sub-task of ...,0.195207
6,Automatic extraction of drum tracks from polyp...,We propose an approach for extracting automati...,0.167280
0,Automatic drum transcription and source separa...,While research has been carried out on automat...,0.162654
15,Automatic transcription of drum sequences usin...,The transcription of a musical performance fro...,0.153321
17,A review of automatic drum transcription,"In Western popular music, drums and percussion...",0.133598


In [12]:
# get the 50 papers with highest similarity
recomPaper50 = recomPaper[:51]

In [13]:
recomPaper50.shape

(28, 3)

In [15]:
recomPaper50 = recomPaper50.dropna().reset_index(drop=True)

recomPaper50.to_csv("recomPapers50.csv")